##introduccion

En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

##instalacion de librerias

In [2]:
!pip install demoji

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install memory-profiler

Defaulting to user installation because normal site-packages is not writeable


## Import

In [4]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [5]:
import json
import demoji
import pandas as pd
from datetime import datetime
from typing import List, Tuple
from collections import Counter
from collections import defaultdict

descarga los emojis

In [6]:
demoji.download_codes()

C:\Users\kullo\AppData\Local\Temp\ipykernel_24400\2299328559.py:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


##Creacion Dataframe

In [7]:
registros = []

try:
    with open(file_path, 'r') as archivo:
        for num_linea, linea in enumerate(archivo, start=1):
            # Carga el objeto JSON de cada línea
            try:
                registro = json.loads(linea)
                registros.append(registro)
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en la línea {num_linea}: {linea.strip()}")
                print(e)
except FileNotFoundError:
    print(f"Error: El archivo {file_path} no fue encontrado.")
except Exception as general_error:
    print(f"Error inesperado: {general_error}")

# Convierte la lista de registros a un DataFrame
if registros:
    df = pd.DataFrame(registros)
    print("DataFrame creado con éxito.")
else:
    print("No se encontraron registros válidos para crear el DataFrame.")

DataFrame creado con éxito.


##top 10 fechas donde hay más tweets

In [8]:
def get_top_users_by_date(data: pd.DataFrame) -> List[Tuple[datetime.date, str]]:
    date_user_count = {}

    try:
        for index, entry in data.iterrows():
            # Manejo de errores al convertir la fecha
            try:
                date_str = entry.get('date', '').split('T')[0]
                date = datetime.strptime(date_str, '%Y-%m-%d').date()
            except (ValueError, TypeError) as date_error:
                print(f"Error al convertir la fecha en la fila {index}: {date_error}")
                continue

            # Manejo de errores al acceder al usuario
            try:
                user = entry.get('user', {}).get('username', '')
            except (KeyError, AttributeError) as user_error:
                print(f"Error al acceder al nombre de usuario en la fila {index}: {user_error}")
                continue

            if date not in date_user_count:
                date_user_count[date] = Counter()
            date_user_count[date][user] += 1

    except Exception as general_error:
        print(f"Error inesperado: {general_error}")
        return []

    top_users_by_date = [(date, user_count.most_common(1)[0][0]) for date, user_count in date_user_count.items()]
    return sorted(top_users_by_date, key=lambda x: x[0], reverse=True)[:10]


In [9]:
get_top_users_by_date(df)

[(datetime.date(2021, 2, 24), 'preetysaini321'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 22), 'preetysaini321'),
 (datetime.date(2021, 2, 21), 'Surrypuria'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 15), 'jot__b')]

###medidor de memoria linea por linea Q1

In [10]:
%load_ext memory_profiler
%run q1_memory.py
%run q1_time.py
%mprun -f Q1_memory Q1_memory(df)

###medidor de tiempo Q1

In [11]:
%timeit -r 1 -n 1 q1_time(df)

6.59 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
%time q1_time(df)

CPU times: total: 6.44 s
Wall time: 6.5 s


[(datetime.date(2021, 2, 24), 'preetysaini321'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 22), 'preetysaini321'),
 (datetime.date(2021, 2, 21), 'Surrypuria'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 15), 'jot__b')]

##top 10 emojis más usados con su respectivo conteo

In [13]:
def extract_emojis(text: str) -> List[str]:
    try:
        return list(demoji.findall(text).keys())
    except Exception as extraction_error:
        print(f"Error al extraer emojis: {extraction_error}")
        return []

def get_top_emojis(data: pd.DataFrame) -> List[Tuple[str, int]]:
    emoji_counter = Counter()

    try:
        for _, entry in data.iterrows():
            # Manejo de errores al acceder al contenido
            try:
                content = entry.get('content', '')
            except (KeyError, AttributeError) as content_error:
                print(f"Error al acceder al contenido: {content_error}")
                continue

            emojis = extract_emojis(content)
            emoji_counter.update(emojis)

    except Exception as general_error:
        print(f"Error inesperado: {general_error}")
        return []

    return emoji_counter.most_common(10)

In [14]:
get_top_emojis(df)

[('🙏', 3203),
 ('😂', 1387),
 ('🚜', 1334),
 ('🌾', 1298),
 ('❤️', 1205),
 ('✊', 1110),
 ('🇮🇳', 938),
 ('🤣', 759),
 ('👍', 634),
 ('🙏🏻', 580)]

###medidor de memoria linea por linea Q2

In [ ]:
%load_ext memory_profiler
%run q2_memory.py
%run q2_time.py
%mprun -f Q2_memory Q2_memory(df)

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


###medidor de tiempo Q2

In [ ]:
%timeit -r 1 -n 1 q2_time(df)


In [ ]:
%time q2_time(df)

##top 10 histórico de usuarios (username) más influyentes en función del conteo de las menciones (@) que registra cada uno de ellos

In [ ]:
def top_influential_users(data: pd.DataFrame) -> List[Tuple[str, int]]:
    mention_count = defaultdict(int)

    try:
        for index, row in data.iterrows():
            # Manejo de errores al acceder a mentionedUsers
            try:
                mentioned_users = row.get("mentionedUsers")
            except (KeyError, AttributeError) as mentioned_users_error:
                print(f"Error al acceder a mentionedUsers en la fila {index}: {mentioned_users_error}")
                continue

            # Verificar si mentioned_users no es nulo
            if mentioned_users is not None:
                for user in mentioned_users:
                    # Manejo de errores al acceder a username
                    try:
                        username = user.get("username")
                    except (KeyError, AttributeError) as username_error:
                        print(f"Error al acceder a username en la fila {index}: {username_error}")
                        continue

                    if username is not None:
                        mention_count[username] += 1

    except Exception as general_error:
        print(f"Error inesperado: {general_error}")
        return []

    top_users = sorted(mention_count.items(), key=lambda x: x[1], reverse=True)[:10]
    return top_users

In [ ]:
# Obtener el top 10
top_10_users = top_influential_users(df)

# Mostrar el resultado
top_10_users

###medidor de memoria linea por linea Q3

In [ ]:
%load_ext memory_profiler
%run q3_memory.py
%run q3_time.py
%mprun -f Q3_memory Q3_memory(df)

###medidor de tiempo Q3

In [ ]:
%timeit -r 1 -n 1 q3_time(df)

In [ ]:
%time q3_time(df)